# TTree interactive browsing
* This is prototype written as a  "macro" set of function
  * example not yet finished
** Direct usage of ROOT and AliROOT libSTAT functionality
* Functionality to be encapuslated
* Functions:
  * Init "libraries"
  * Load tree
  * Load widgets
  * Draw widgets
* Example using tracking performance parameterization 


# Init tree browsing 
* Load widgets for interaction with root objects
* Enable ROOT
* Load AliROOT libStat

In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipywidgets import Button, Layout
import numpy as np
from IPython.display import display
import ROOT
from RootInteractive.Tools.aliTreePlayer import *
ROOT.gSystem.Load("$ALICE_ROOT/lib/libSTAT.so")
%jsroot

In [ ]:
c1=ROOT.TCanvas("c1","c1",600,400);

### Load trees from the list (e.g.)
* Load example trees for the TRD tracking comissioning
  * pt spectra and covaraince matrix of q/pt for 2 differnt reconstruction passes + anchor MC
  * 3 different trigger configuration (TMB, TCalo, TEJ1)
  * 3 differnt covariance matrix cut (05,15,30)
  * 3 different pt depenedent cut on the number of TPC crossed rows (80,100,120)
  *  different pt depenedent cut on the number of TPC crossed rows + TRD clusters(80,100,120)
* Total  120 trees 

In [ ]:
tree, treeList, fileList = LoadTrees("cat mapLong.list","his.*_proj_0_1Dist","$#",".*",0)
tree.SetAlias("medianMean","median-mean")
friendListROOT = tree.GetListOfFriends();
branchListROOT = tree.GetListOfBranches();
aliasListROOT = tree.GetListOfAliases();

### Load list of friends, branches and aliases
* To do  - add protection ageain 0 pointers 

In [ ]:
friendList=['']
branchList=[]
aliasList=['']
sliderWidgets=[]
friendMask='.*05.*TMB.*'
branchMask='.*rms.*'
print('Total number of trees',tree.GetListOfFriends().GetEntries());
print('Selecting subset of friends', friendMask)

In [ ]:
friendMaskWidget=widgets.Text(description='Friend mask', value=friendMask);
branchMaskWidget=widgets.Text(description='Branch mask',value=branchMask);
aliasMaskWidget=widgets.Text(description='Alias mask');
friendDropDown=widgets.Dropdown( options=friendList, description='Friend list:');
branchDropDown=widgets.Dropdown( options=branchList, description='Branch list:');
aliasDropDown=widgets.Dropdown( options=aliasList, description='Alias list:');
maskButton=widgets.Button(description='Mask',tooltip="Mask");
addQueryButton=widgets.Button(description='Add Query',tooltip="Add query");
addSelectionButton=widgets.Button(description='Add selection',tooltip="Add selection");
addSliderButton=widgets.Button(description='Add slider',tooltip="Add slider");
#
drawQuery=widgets.Text(description='Query',layout=Layout(width='99%'));
drawCut=widgets.Text(description='Selection',layout=Layout(width='99%'));
drawSlider=widgets.Text(description='Slider',layout=Layout(width='99%'));

In [ ]:
def funUpdateList():
    #friendMask=friendMaskWidget.value
    friendMaskReg=ROOT.TPRegexp(friendMask)
    branchMaskReg=ROOT.TPRegexp(branchMask)
    friendList[:]=['']
    for i in range(0,friendListROOT.GetEntries()):
        if friendMaskReg.GetPattern().Length() > 0:
            if friendMaskReg.Match(friendListROOT.At(i).GetName()):
                friendList.append(friendListROOT.At(i).GetName())
        else:
            friendList.append(friendListROOT.At(i).GetName())
    branchList[:]=['']       
    for i in range(0,branchListROOT.GetEntries()): 
        if branchMaskReg.GetPattern().Length() > 0:
            if branchMaskReg.Match(branchListROOT.At(i).GetName()):
                branchList.append(branchListROOT.At(i).GetName())
        else:    
            branchList.append(branchListROOT.At(i).GetName())  
    for i in range(0,aliasListROOT.GetEntries()): 
        aliasList.append(aliasListROOT.At(i).GetName())
    friendDropDown.options=friendList
    branchDropDown.options=branchList
def funAddQuery(b): 
    drawQuery.value+=friendDropDown.value
    drawQuery.value+='.'
    drawQuery.value+=branchDropDown.value
def funAddSelection(b): 
    drawCut.value+=friendDropDown.value
    drawCut.value+='.'
    drawCut.value+=branchDropDown.value    
def funUpdateMask(b):
    global branchMask
    global friendMask
    branchMask=str(branchMaskWidget.value)
    friendMask=str(friendMaskWidget.value)
    #print(branchMask)
    funUpdateList()

## Draw widget

In [ ]:
funUpdateList()
boxMask=widgets.HBox([friendMaskWidget,branchMaskWidget, aliasMaskWidget,maskButton]);
boxSelect=widgets.HBox([friendDropDown,branchDropDown, aliasDropDown]);
boxButton=widgets.HBox([addQueryButton,addSelectionButton,addSliderButton]);
boxDraw=widgets.VBox([drawQuery,drawCut,drawSlider], width=1000);
boxSlider=widgets.VBox(sliderWidgets);
#
boxAll=widgets.VBox([boxMask,boxSelect,boxButton,boxDraw,boxSlider])
#
addQueryButton.on_click(funAddQuery)
addSelectionButton.on_click(funAddSelection)
maskButton.on_click(funUpdateMask)
branchMaskWidget.observe(funUpdateMask,branchMaskWidget.value) # looks liek not working
boxAll


# Debugging part
* printing widget content


In [ ]:
branchMask='.*rms.*'
print(branchMask)
print(branchMaskWidget)
funUpdateMask(maskButton)
print(branchMask)

In [ ]:
branchMask='xxx'
print(branchMaskWidget)
print(branchMask)
branchMaskReg=ROOT.TPRegexp(branchMask)
branchMask=str(branchMaskWidget.value)
print(branchMask)
branchMaskReg=ROOT.TPRegexp(branchMask)


In [ ]:
tree.Show(0)